In [1]:
class Struct(object):
    def __init__(self, kwargs):
        for k, v in kwargs.items():
            self.__dict__[k] = Struct(v) if isinstance(v, dict) else v
            
settings = Struct({'A': 0,
 'D': 0,
 'H': (0.0, 0.0, 100.0),
 'Hdir': (0, 0, 1),
 'Hmag': 100.0,
 'K': 0,
 'Ms': 800000.0,
 'alpha': 0.8,
 'correction': 0,
 'e': (1, 0, 0),
 'frames': 10,
 'gamma0': 221100.0,
 'grid': {'d': (1e-09, 1e-09, 1e-09),
  'l': (1.0000000000000001e-07, 5.0000000000000004e-08, 1e-08),
  'n': (100, 50, 10)},
 'init': 'flower',
 'mu0': 1.2566370614e-06,
 'periodic_boundary': True,
 'save_every': 100,
 'time': {'d': 1e-14, 'l': 1e-11, 'n': 1000},
 'value': None})     

In [3]:
import os
import sys

if not "DEVITO_OPENMP" in os.environ or os.environ["DEVITO_OPENMP"] != "1":
    print("*** WARNING: Devito OpenMP environment variable has not been set ***", file=sys.stderr)
os.environ['OMP_NUM_THREADS'] = "8"

import numpy as np
from sympy import Matrix, Eq, solve
from devito import Grid, Function, TimeFunction, configuration, Operator

def vector_laplacian(u):
    return Matrix([u[0].dx2 + u[0].dy2 + u[0].dz2,
                   u[1].dx2 + u[1].dy2 + u[1].dz2,
                   u[2].dx2 + u[2].dy2 + u[2].dz2])

def vector_gradient(u):
    return u[0].dx**2 + u[0].dy**2 + u[0].dz**2 + u[1].dx**2 + u[1].dy**2 + u[1].dz**2 + u[2].dx**2 + u[2].dy**2 + u[2].dz**2

def curl(u):
    return Matrix([u[2].dy - u[1].dz,
                   u[0].dz - u[2].dx,
                   u[1].dx - u[0].dy])

def VectorTimeFunction(name, settings):
    return Matrix([TimeFunction(name='{}_x'.format(name), **settings), TimeFunction(name='{}_y'.format(name), **settings), TimeFunction(name='{}_z'.format(name), **settings)])


def VectorDenseData(name, settings):
    return Matrix([Function(name='{}_x'.format(name), **settings), Function(name='{}_y'.format(name), **settings), Function(name='{}_z'.format(name), **settings)])

buffer_params = Struct({'buffer_dims': (102, 52, 12),
 'buffer_slice': (slice(1, -1, None), slice(1, -1, None), slice(1, -1, None)),
 'data_dims': (100, 50, 10)})

In [6]:
RKc = [[0, 0, 0], [0.5, 0, 0], [2, 1, 0], [1/6, 2/3, 1/6]]

grid = Grid(shape=buffer_params.buffer_dims, extent=settings.grid.l)
m = VectorTimeFunction('m', {"grid":grid, "space_order":2, "time_order":3})
k = VectorTimeFunction('k', {"grid":grid, "space_order":2, "time_order":3})
kc = TimeFunction(name='kc', **{"grid":Grid(shape=(3,)), "space_order":1, "time_order":3})

kc.data[:] = np.array(RKc)

nx, ny, nz = buffer_params.buffer_dims
dt = settings.time.d
update = []

c = 2 / (settings.mu0 * settings.Ms)
e = Matrix(settings.e)

zeeman = Matrix(settings.Hdir) * settings.Hmag
exchange = settings.A * c * vector_laplacian(m)
anisotropy = settings.K * c * m.dot(e) * e
dmi = settings.D * c * curl(m)
heff = zeeman + exchange + anisotropy + dmi

crossHeff = m.cross(heff)
LLG = -settings.gamma0 / (1 + settings.alpha**2) * (crossHeff + settings.alpha * m.cross(crossHeff))

x, y, z = grid.dimensions
t = grid.stepping_dim

for ki, llgi in zip(k, LLG):
    update.append(Eq(ki.indexed[t + 1, x, y, z], llgi))
    if settings.periodic_boundary:
        update.append(Eq(ki.indexed[t + 1, x, y, 0], ki.indexed[t + 1, x, y, nz - 2]))
        update.append(Eq(ki.indexed[t + 1, x, y, nz - 1], ki.indexed[t + 1, x, y, 1]))
        update.append(Eq(ki.indexed[t + 1, x, 0, z], ki.indexed[t + 1, x, ny - 2, z]))
        update.append(Eq(ki.indexed[t + 1, x, ny - 1, z], ki.indexed[t + 1, x, 1, z]))
        update.append(Eq(ki.indexed[t + 1, 0, y, z], ki.indexed[t + 1, nx - 2, y, z]))
        update.append(Eq(ki.indexed[t + 1, nx - 1, y, z], ki.indexed[t + 1, 1, y, z]))
    else:
        update.append(Eq(ki.indexed[t + 1, x, y, 0], 0.))
        update.append(Eq(ki.indexed[t + 1, x, y, nz - 1], 0.))
        update.append(Eq(ki.indexed[t + 1, x, 0, z], 0.))
        update.append(Eq(ki.indexed[t + 1, x, ny - 1, z], 0.))
        update.append(Eq(ki.indexed[t + 1, 0, y, z], 0.))
        update.append(Eq(ki.indexed[t + 1, nx - 1, y, z], 0.))
    
for mi, ki in zip(m, k):
    update.append(Eq(mi.indexed[t + 1, x, y, z], mi.indexed[t, x, y, z] + kc.indexed[t + 1, 0] * ki.indexed[t + 1, x, y, z] + kc.indexed[t, 1] * ki.indexed[t, x, y, z] + kc.indexed[t - 1, 2] * ki.indexed[t - 1, x, y, z]))

for u in update:
    print(u, file=sys.stderr)
op = Operator(update)
print(op.ccode, file=sys.stderr)

Eq(k_x[t + 1, x, y, z], -10785365.8536585*m_x(t, x, y, z)*m_z(t, x, y, z) - 13481707.3170732*m_y(t, x, y, z))
Eq(k_x[t + 1, x, y, 0], k_x[t + 1, x, y, 10])
Eq(k_x[t + 1, x, y, 11], k_x[t + 1, x, y, 1])
Eq(k_x[t + 1, x, 0, z], k_x[t + 1, x, 50, z])
Eq(k_x[t + 1, x, 51, z], k_x[t + 1, x, 1, z])
Eq(k_x[t + 1, 0, y, z], k_x[t + 1, 100, y, z])
Eq(k_x[t + 1, 101, y, z], k_x[t + 1, 1, y, z])
Eq(k_y[t + 1, x, y, z], 13481707.3170732*m_x(t, x, y, z) - 10785365.8536585*m_y(t, x, y, z)*m_z(t, x, y, z))
Eq(k_y[t + 1, x, y, 0], k_y[t + 1, x, y, 10])
Eq(k_y[t + 1, x, y, 11], k_y[t + 1, x, y, 1])
Eq(k_y[t + 1, x, 0, z], k_y[t + 1, x, 50, z])
Eq(k_y[t + 1, x, 51, z], k_y[t + 1, x, 1, z])
Eq(k_y[t + 1, 0, y, z], k_y[t + 1, 100, y, z])
Eq(k_y[t + 1, 101, y, z], k_y[t + 1, 1, y, z])
Eq(k_z[t + 1, x, y, z], 10785365.8536585*m_x(t, x, y, z)**2 + 10785365.8536585*m_y(t, x, y, z)**2)
Eq(k_z[t + 1, x, y, 0], k_z[t + 1, x, y, 10])
Eq(k_z[t + 1, x, y, 11], k_z[t + 1, x, y, 1])
Eq(k_z[t + 1, x, 0, z], k_z[t + 1,